In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# print(os.getenv("OPENAI_API_KEY"))

from langchain.graphs import Neo4jGraph
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Neo4jVector

graph = Neo4jGraph()

In [ ]:
class PrintRetrievedText:
    def __call__(self, data):
        print(data)
        if isinstance(data, dict):
            context = data.get("context", {})
            retrieved_text = context.get("text")
            if retrieved_text:
                print("Retrieved Text:", retrieved_text)
            print(data)
        return data

In [ ]:
def query_case(entity_title, question, method):

    mentioned_in_query = """
    MATCH (p:Person)-[:MENTIONED_IN]->(e:Extract)
    WHERE p.name = '""" + entity_title + """'
    WITH e, max(score) AS score 
    RETURN e.text AS text, score, {} AS metadata
    """

    print("Searching for entity:" + mentioned_in_query)

    hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
        OpenAIEmbeddings(),
        index_name="hypothetical_questions",
        retrieval_query=mentioned_in_query,
    )

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    model = ChatOpenAI()

    retriever = hypothetic_question_vectorstore.as_retriever()

    print(retriever.get_relevant_documents("Who were the guarantors?"))

    chain = (
        RunnableParallel(
            {
                "context": itemgetter("question") | retriever,
                "question": itemgetter("question"),
            }
        )
        | prompt
        | model
        | StrOutputParser()
        | PrintRetrievedText()
    )

    return chain.invoke({"question": question})

In [ ]:
query_case("[2023] SGHC(I) 21", question, "QUESTION")